# Активный словарь русского языка

In [1]:
import re
import os
import json
import pickle
from collections import Counter
from tqdm.auto import tqdm
from pypdf import PdfReader, PdfWriter
from transliterate import translit

## Шаг 0: подготовка к распознаванию

In [2]:
act1 = PdfReader('Активный_словарь_русского_языка_том_1.pdf')  # 37-403
act2 = PdfReader('Активный_словарь_русского_языка_том_2.pdf')  # 4-737
act3 = PdfReader('Активный_словарь_русского_языка_том_3.pdf')  # 3-768

In [3]:
outfile = PdfWriter()

for pageNum in range(36, 403):
    page = act1.pages[pageNum]
    outfile.add_page(page)
for pageNum in range(3, 737):
    page = act2.pages[pageNum]
    outfile.add_page(page)
for pageNum in range(2, 768):
    page = act3.pages[pageNum]
    outfile.add_page(page)

In [4]:
with open('active_dictionary.pdf', 'wb') as f:
    outfile.write(f)

## Шаг 1: OCR с помощью ABBYY FineReader
## Шаг 2: извлечение текста из файла

In [5]:
reader = PdfReader('active_dictionary_with_ocr.pdf')

In [6]:
pages_text = []
for page in tqdm(reader.pages):
    pages_text.append(page.extract_text())

  0%|          | 0/1867 [00:00<?, ?it/s]

In [11]:
# мелкая ручная очистка
pages_text[0] = pages_text[0][2:]
pages_text[94] = pages_text[94][2:]
pages_text[367] = pages_text[367][2:]
pages_text[919] = pages_text[919][2:]
pages_text[1101] = pages_text[1101][2:]
pages_text[1414] = pages_text[1414][2:]
pages_text[1450] = pages_text[1450][2:]
pages_text[1515] = pages_text[1515][2:]

pages_text[-1] = pages_text[-1].split('\nНаучное  издание\n')[0]

In [12]:
# сохраняем в пикл для экономии времени при дальнейшних загрузках
with open('active_dict_pages.pickle', 'wb') as outputfile:
    pickle.dump(pages_text, outputfile)

## Шаг 3: разбиение на словарные статьи

In [2]:
with open('active_dict_pages.pickle', 'rb') as outputfile:
    pages_text = pickle.load(outputfile)

len(pages_text)

1867

In [3]:
# убираем номера и заголовки страниц
filt = [re.sub(r'^[А-ЯAЁ]+ \d+ [А-ЯAЁ]+', '', page) for page in pages_text]

### Очистка текста

In [4]:
rep_dict = {
    '\nУ Употребляется в функции СУЩ': '\nУпотребляется в функции СУЩ',
    '\nАЛМА з': '\nАЛМАЗ',
    '\nАМБА р': '\nАМБАР',
    '\nможно в сочетании без толку] и БЕЗО': 'можно в сочетании без толку] и БЕЗО',
    '\nБИЛЕ т': '\nБИЛЕТ',
    ' большие 2.2': '\nбольшие 2.2',
    '\nвгляд Еться': '\nВГЛЯДЕТЬСЯ',
    ' влюбленные 1.2': '\nвлюбленные 1.2',
    '\nАНА: как таковой; в чистом виде; как абстракция; в целом. II': '\nАНА: как таковой; в чистом виде; как абстракция; в целом.\nII',
    'ВОСПРОИЗВЕСТИ. ВОССТАВАТЬ': 'ВОСПРОИЗВЕСТИ.\nВОССТАВАТЬ',
    'см. ВОЙТИ.ВЧЕРА': 'см. ВОЙТИ.\nВЧЕРА',
    'ла (М. Ростропович).II': 'ла (М. Ростропович).\nII',
    ' глухонемой 2': '\nглухонемой 2',
    '[обычно в 1.1, 1.2, 2, 3.1, 3.2]\n, НАРЕЧ;': '[обычно в 1.1, 1.2, 2, 3.1, 3.2], НАРЕЧ;',
    'см. горбатый 1.2.ГОРБАТЫЙ': 'см. горбатый 1.2.\nГОРБАТЫЙ',
    'горбатиться. горбатый 1.2': 'горбатиться.\nгорбатый 1.2',
    'ДАЛЁКО [кроме 3]\n, НАРЕЧ;': 'ДАЛЁКО [кроме 3], НАРЕЧ;',
    '\n. датский 2,': '.\nдатский 2,',
    'ДЕ ВИЦА [только 2]\n, СУЩ;': 'ДЕВИЦА [только 2], СУЩ;',
    'АНТ: якобы.действительно 2': 'АНТ: якобы.\nдействительно 2',
    'ист. децимация. десятая 2.2': 'ист. децимация.\nдесятая 2.2',
    'ДИТЯ!, СУЩ; СРЕДН, ОДУШ;': 'ДИТЯ, СУЩ; СРЕДН, ОДУШ;',
    '\nДОБРО), СУЩ; СРЕДН;': '\nДОБРО, СУЩ; СРЕДН;',
    'Обращение дорогой, дорогая без СУЩ': 'Обращение дорогой, дорогая без сущ.',
    '\nДОСКА., СУЩ; ЖЕНСК;': '\nДОСКА, СУЩ; ЖЕНСК;',
    '\n.ДУЭТ, СУЩ; МУЖСК;': '\nДУЭТ, СУЩ; МУЖСК;',
    '\nЕГО), ПРИЛ; МЕСТ, ПРИТЯЖ;': '\nЕГО, ПРИЛ; МЕСТ, ПРИТЯЖ;',
    '\nЖЮРИ!, СУЩ; СРЕДН;': '\nЖЮРИ, СУЩ; СРЕДН;',
    'брать <хватать> за сердце]\n': 'брать <хватать> за сердце] ',
    'см. ЗАБЕЖАТЬ.ЗАБЕЖАТЬ': 'см. ЗАБЕЖАТЬ.\nЗАБЕЖАТЬ',
    'см. ЗАВЕРШИ!ТЬ.ЗАВЕРШИТЬ': 'см. ЗАВЕРШИТЬ.\nЗАВЕРШИТЬ',
    '\nЗАВЕСТИ!, ГЛАГ;': '\nЗАВЕСТИ, ГЛАГ;',
    '\nЗАНЕСТИ!, ГЛАГ;': '\nЗАНЕСТИ, ГЛАГ;',
    '\nЗАОДНО), НАРЕЧ.': '\nЗАОДНО, НАРЕЧ.',
    '\nЗАПАДНЯ!, СУЩ; ЖЕНСК;': '\nЗАПАДНЯ, СУЩ; ЖЕНСК;',
    '\nЗАПАСТИ!, ГЛАГ;': '\nЗАПАСТИ, ГЛАГ;',
    '\nЗАРАСТИ!, ГЛАГ;': '\nЗАРАСТИ, ГЛАГ;',
    '\nЗАРЯ!, СУЩ; ЖЕНСК;': '\nЗАРЯ, СУЩ; ЖЕНСК;',
    '\nЗАТО), СОЮЗ;': '\nЗАТО, СОЮЗ;',
    '\nЗВЕНО), СУЩ; СРЕДН;': '\nЗВЕНО, СУЩ; СРЕДН;',
    '\nЗЕРНО), СУЩ; СРЕДН;': '\nЗЕРНО, СУЩ; СРЕДН;',
    '\nЗМЕЯ!, СУЩ, ЖЕНСК;': '\nЗМЕЯ, СУЩ, ЖЕНСК;',
    'на вес золота. золотой 4': 'на вес золота.\nзолотой 4',
    'НЕСОВ; см. ВТЕ РЕТЬСЯ.ВТИХОМОЛКУ': 'НЕСОВ; см. ВТЕРЕТЬСЯ.\nВТИХОМОЛКУ',
    'СОВ; см. ВТОР ГАТЬСЯ.ВТОРИЧНО': 'СОВ; см. ВТОРГАТЬСЯ.\nВТОРИЧНО',
    'НЕСОВ; см. ВЫБРАТЬ.ВЫБИРАТЬСЯ': 'НЕСОВ; см. ВЫБРАТЬ.\nВЫБИРАТЬСЯ',
    'НЕСОВ; см. ВЫПРО СИТЬ.ВЫПРОСИТЬ': 'НЕСОВ; см. ВЫПРОСИТЬ.\nВЫПРОСИТЬ',
    'ДЕВЯТН АДТЦА ТЫЙ. ПРИЛ; ПОРЯДК;': 'ДЕВЯТНАДТЦАТЫЙ, ПРИЛ; ПОРЯДК;',
    'НЕСОВ; см, ЗАЖАТЬ,ЗАЖИТЬ': 'НЕСОВ; см. ЗАЖАТЬ.\nЗАЖИТЬ',
}

In [5]:
def clean_pages(pages):
    merged = '\n' + '\n'.join(pages)
    m_clean = merged.replace('\xad', '')
    m_clean = re.sub(r' +', ' ', m_clean)  # схлопываем повторяющиеся пробелы
    m_clean = re.sub(r'\s([,;.’\]])', '\g<1>', m_clean)  # убираем пробелы перед знаками препинания
    m_clean = m_clean.replace(']', ']\n').replace(']\n ', '] ').replace(' \n', ' ')
    m_clean = re.sub(r'\n+', '\n', m_clean)  # схлопываем повторяющиеся переводы строки
    m_clean = re.sub(
        ', (ГЛАГ|МЕЖДОМ|НАРЕЧ|ПРЕДЛОГ|ПРИЛ|СОЮЗ|СУЩ|ЧАСТ|ЧИСЛ|глаг|междом|нареч|предлог|прил|союз|сущ|част|числ)(,|;)',
        lambda m: f', {m.group(1).upper()}{m.group(2)}', m_clean)

    for key, val in list(rep_dict.items()):
        m_clean = m_clean.replace(key, val)

    return m_clean

In [6]:
m_clean = clean_pages(filt)
m_clean[0:1000]

'\nA, СОЮЗ; СОЧИН.\na 1.1 указывает на то, что первая ситуация обычно препятствует существованию второй: Пообедал, а почему-то есть хочется.\na 1.2 указывает на то, что вторая ситуация, уже известная адресату, обычно препятствует существованию первой: Так есть хочется, а ведь только что пообедал.\na 1.3 указывает на то, что субъект второй ситуации не учитыва ет существования первой: День был дождливый, а Петя не взял зонт.\na 1.4 указывает на сосуществование двух ситуаций, которые обычно не существуют одновременно, так как фактор, который обусловли вает первую ситуацию, одновременно препятствует существованию второй: Солнце каждый день садится в тучу, а дождя все нет.\na 1.5 указывает на то, что вторая ситуация препятствует выполнению желаемого: Хотел поехать в круиз, а билетов уже не было.\na 2 указывает на различие объектов по общему для них признаку: Я живу в Москве, а он в Лондоне; Коля толстый, а Ваня худой.\na 3.1 указывает на то, что в фокусе внимания говорящего - упоми навшийся

In [7]:
# for i, page in enumerate(filt):
#     if 'звонить' in page:
#         print(i)
# clean_pages([filt[1844]])

### Выделение заголовков словарных статей

In [8]:
pos_markers = ['ГЛАГ', 'МЕЖДОМ', 'НАРЕЧ', 'ПРЕДЛОГ',
               'ПРИЛ', 'СОЮЗ', 'СУЩ', 'ЧАСТ', 'ЧИСЛ']
SPLITTER = '(\n.+(' + '|'.join(pos_markers) + '))'
SPLITTER

'(\n.+(ГЛАГ|МЕЖДОМ|НАРЕЧ|ПРЕДЛОГ|ПРИЛ|СОЮЗ|СУЩ|ЧАСТ|ЧИСЛ))'

In [9]:
articles = re.findall(SPLITTER, m_clean)
len(articles)

3268

In [10]:
safe_to_filter = {
    'беспризорный 1.1',
    'бессовестный 1',
    'бессознательное 3',
    'бесстыжий 1.1',
    'бесстыжий 2.1',
    'бис 1',
    'бис 2',
    'божественный 2',
    'более 3.2',
    'большие 2.2',
    'взамен 1',
    'взамен 2',
    'винт 3',
    'влюбленные 1.2',
    'восемьдесят 3',
    'впереди 3.2',
    'все равно 1',
    'все равно 2',
    'глухой 1.2',
    'глухонемой 2',
    'гнедой 1',
    'горбатый 1.2',
    'грязный 2.2',
    'грязное 2.3',
    'датский 2',
    'двадцатое 1.2',
    'двадцатая 2.2',
    'двое 2.1',
    'двое 2.2',
    'девятнадцатое 1.2',
    'девятнадцатая 2.2',
    'девятое 1.2',
    'девятая 2.2',
    'девять 4',
    'действительно 1',
    'действительно 2',
    'деревянные 3',
    'десятое 1.2',
    'десятая 2.2',
    'детская 2.2',
    'домашние 3',
    'древние 1.2',
    'другой 3',
    'другое 4',
    'дуэтом 3, только в форме ТВОР ЕД',
    'едва 1.1',
    'едва 1.2',
    'едва 2.1',
    'едва 2.2',
    'единичное 4',
    'единый 3.2',
    'жалко 3',
    'завтра 1.1',
    'завтра 1.2',
    'завтра 2.2',
    'заграничное 2',
    'зеленые 5',
    'зеленые 6.2',
    'зеленые 7',
    'злое 3.2',
    'знакомый 4',
    'золотой 4',
    'зубной 4',
}

In [11]:
splitters = []

for art in articles:
    art_title = art[0]
    no_pos_parts = art_title.split(', ')[0:-1]
    if no_pos_parts:
        no_pos = ', '.join(no_pos_parts).strip()
        if no_pos not in {'I', 'II'}:
            final_title = re.sub(', (ГЛАГ|МЕЖДОМ|НАРЕЧ|ПРЕДЛОГ|ПРИЛ|СОЮЗ|СУЩ|ЧАСТ|ЧИСЛ).*?', '', no_pos)
            if re.search('^[А-ЗЁABEа-зёae]', final_title) and final_title not in safe_to_filter:
                splitters.append(art_title)

len(splitters)

3099

In [12]:
add_arts = re.findall('\n[А-ЗЁABE][А-Я]+\n', m_clean)
len(add_arts)

16

In [13]:
titles = splitters + add_arts
len(titles)

3115

In [14]:
titles[0:10]

['\nA, СОЮЗ',
 '\nАБРИКОС, СУЩ',
 '\nАБРИКОСОВЫЙ, ПРИЛ',
 '\nАБСОЛЮТНЫЙ, ПРИЛ',
 '\nАБСТРАКТНЫЙ, ПРИЛ',
 '\nАБСУРД, СУЩ',
 '\nАБСУРДНЫЙ, ПРИЛ',
 '\nАВАНГАРД, СУЩ',
 '\nАВАНС, СУЩ',
 '\nАВАНТЮРА, СУЩ']

### Разбиение на словарные статьи
[re.split](https://stackoverflow.com/questions/2136556/in-python-how-do-i-split-a-string-and-keep-the-separators)

In [15]:
big_splitter = 'SPECIAL_JOINING_EXPRESSION'.join(titles)
escape_chars = '\.+*?^$()[]{}|'
for char in escape_chars:
    big_splitter = big_splitter.replace(char, f'\{char}')
big_splitter = big_splitter.replace('SPECIAL_JOINING_EXPRESSION', '|')
big_splitter = f'({big_splitter})'

In [16]:
sep_arts = re.split(big_splitter, m_clean)[1:]
len(sep_arts)

6216

In [17]:
merged_arts = {}
for i in range(0, len(sep_arts) - 1, 2):
    merged_arts[sep_arts[i]] = sep_arts[i + 1]

In [18]:
manual_pos = {
    'АБОРДАЖ': 'СУЩ',
    'БАКЛУШИ': 'СУЩ',
    'БЕССТЫДНЫЙ': 'ПРИЛ',
    'БРУДЕРШАФТ': 'СУЩ',
    'ВОЗМУЩАТЬСЯ': 'ГЛАГ',
    'ДА': 'ЧАСТ, СОЮЗ и МЕЖДОМ',
    'ДАВАТЬ': 'ГЛАГ',
    'ДАВИТЬ': 'ГЛАГ',
    'ДАВНО': 'НАРЕЧ',
    'ДАР': 'СУЩ',
    'ДАТСКИЙ': 'ПРИЛ',
    'ДВОЙНИК': 'СУЩ',
    'ДОМОЙ': 'НАРЕЧ',
    'ДРУЖНЫЙ': 'ПРИЛ',
    'ЗАВЛАДЕТЬ': 'ГЛАГ',
    'ЗАПУТАННЫЙ': 'ПРИЛ',
}

In [19]:
def clean_lemma(lemma):
    lemma = translit(lemma.lower(), 'ru')
    return re.sub(r'\d|\d\.\d', '', lemma).strip()

In [20]:
dict_info = {}
cleared = []
for title, article in list(merged_arts.items()):
    title_with_pos = title.split(', ')
    lemma = ''.join(title_with_pos[0:-1]).strip()
    pos = title_with_pos[-1].strip()
    if not re.search('ГЛАГ|МЕЖДОМ|НАРЕЧ|ПРЕДЛОГ|ПРИЛ|СОЮЗ|СУЩ|ЧАСТ|ЧИСЛ', pos):
        lemma = pos
        try:
            pos = manual_pos[pos]
        except KeyError:
            manual_pos[pos] = input(f'Введите часть речи для слова {pos}: ').strip()
            pos = manual_pos[pos]
    dict_info[lemma] = {'lemma': clean_lemma(lemma),
                        'pos': pos,
                        'definition': article.strip()}

In [21]:
list(dict_info.items())[-1]

('ЗЯТЬ',
 {'lemma': 'зять',
  'pos': 'СУЩ',
  'definition': '; МУЖСК, ОДУШ; -я, МН -ья, -ёв.\nЗять решил сделать ремонт в квартире; Иван приходится мне зятем.ЗНАЧЕНИЕ. Зять А2 ‘Муж дочери, сестры или внучки чело века А2’.\nУПРАВЛЕНИЕ.\nА1 • РОД: зять Марии Ивановны, зять (наших) друзей.\n• ПРИТЯЖ: мой <наш> зять.\n-J Через некоторое время даже зять привык к подгоревшей картошке (А. Геласимов). Георгий Иванович приглашал дочь с зятем переезжать к ним, на Алтай (Л. Улицкая). Мать Сони питала особую симпатию к Кунику, кажется, видела в нем идеального зятя, но Соня была по этому поводу друго го мнения (Ю. Трифонов). Все чаще ему хотелось исчезнуть из дому, квартира перестала радовать, зять вызывал тош ноту, Люда - раздражение (И. Муравьева). Гуляли Яковлевы всегда шумно, всегда с драками: то братаны сцепятся, то зять с тестем, то кумовья - по старинке - засопят друг на друга (В. Шукшин). Хаим Ягудин приходился дедушке зя\nтем, был женат на его старшей сестре (А. Рыбаков).\nСИН: ласк. или п

## Информация для автоматической оценки

In [22]:
verb_dict = {}
for lemma in dict_info:
    if dict_info[lemma]['pos'] == 'ГЛАГ':
        verb_dict[lemma] = dict_info[lemma]

In [23]:
len(verb_dict)

1073

In [24]:
list(verb_dict.items())[-1]

('ЗУБРИТЬ',
 {'lemma': 'зубрить',
  'pos': 'ГЛАГ',
  'definition': '; зубрю, зубрит; НЕСОВ; СОВ зазубрить. Понять этого я не мог, приходилось механически зубрить; Учителя по-прежнему иногда заставляют просто зубрить. ЗНАЧЕНИЕ. А1 зубрит А2 ‘Человек А1 для того, чтобы запо мнить информацию А2, много раз повторяет А2 или части А2, возможно, не понимая их’.\nУПРАВЛЕНИЕ.\nА1 • ИМ.\nА2 • ВИН: зубрить сопромат <сольфеджио>.\n• ПРЕДЛ: зубрить, на каких ступенях строятся эти ин\nтервалы.\nСОЧЕТАЕМОСТЬ. Механически <бездумно> зубрить, по-школярски зубрить, зубрить наизусть.\nРаскрываю атлас и начинаю зубрить: - Лена. Обь. Енисей (К. Чуковский). Легкость усвоения мною латыни они ничем иным не могли себе объяснить, как только тем, что я зуб\nрю (Ю. Олеша). Он продолжал учиться, все свободное вре\nмя зубря немецкий - язык наиболее вероятного противника (К. Симонов). Не довела ли ее до слез учительница, которая требует, чтобы все зубрили наизусть: «Надо выучить так, чтобы от зубов отскакивало» (Л. 

In [25]:
# with open('active_dict_verbs.pickle', 'wb') as outputfile:
#     pickle.dump(verb_dict, outputfile)

### Активный словарь: лексемы

In [26]:
with open('active_dict_verbs.pickle', 'rb') as outputfile:
    verb_dict = pickle.load(outputfile)

In [27]:
def clean_act_verbs(verb):
    if ' и ' in verb:
        verb = verb.split(' и ')[0]
    verb = verb.replace('ё', 'е').replace('йть', 'ить').replace(
        'шть', 'тить').replace('стй', 'сти').replace(
        'ьть', 'ыть').replace('чйсл', 'числ').replace(
        ' ', '').replace('йск', 'иск').replace('тбся', 'ться').replace(
        'запоолнить', 'заполнить')
    return translit(verb, 'ru')

In [28]:
act_verbs = [val['lemma'] for val in list(verb_dict.values())]
act_verbs = [clean_act_verbs(verb) for verb in act_verbs]

In [29]:
print(f'Глаголов в Активном словаре: {len(act_verbs)}')

Глаголов в Активном словаре: 1073


### Активный словарь: модели управления

In [59]:
act_gov_dict = {}
for verb, info in list(verb_dict.items()):
    v_lemma = clean_act_verbs(info['lemma'])
    act_gov_dict[v_lemma] = re.findall('(\w\d • .+?[:\n])', info['definition'])

In [88]:
case_dict = {
    'ИМ': 'Nom',
    'РОД': 'Gen',
    'ДАТ': 'Dat',
    'ВИН': 'Acc',
    'ТВОР': 'Ins',
    'ПР': 'Loc'}

other_case_set = {'В КАКОМ НАПРАВЛЕНИИ', 'СКОЛЬКО', 'КАКОЕ ВРЕМЯ',
                  'В КАКИХ НАПРАВЛЕНИЯХ', 'КУДА', 'КОГДА', 'НА СКОЛЬКО',
                  'С КАКОЙ ЦЕЛЬЮ', 'ОТКУДА', 'КАК  ЧТО', 'КАКОЙ', 'ТАК',
                  'ГДЕ', 'ЧТОБЫ ИНФ', 'ЗА ЧТО', 'КАК', 'ПО КАКОЙ ПРИЧИНЕ', 'ЧТО'}
filtered_out = []

act_gov_models = {}
for verb in act_gov_dict:
    clean_gov_list = []
    for gov in act_gov_dict[verb]:
        clean_gov = translit(gov, 'ru').strip().strip('.:')
        clean_gov = clean_gov.replace('редк. ', '').replace('редк, ', '').replace('уходящ. ', '').replace('разг. ', '').replace('разг, ', '').replace('МН', '').replace('+', '').replace('«', ' ').replace('»', ' ')
        clean_gov = re.sub('\s+', ' ', re.sub('[<(].*?[>)]', '', clean_gov))
        clean_gov = re.sub('\w\d •', '', clean_gov).replace('Kyn,A', 'КУДА').replace('Кын,А', 'КУДА').replace('ОТКXТА', 'ОТКУДА').split('.')[0]
        clean_gov = f' {clean_gov.strip(" ,")}'

        if re.search(' (ИМ|РОД|ДАТ|ВИН|ТВОР|ПР)$', clean_gov):
            for key, val in list(case_dict.items()):
                clean_gov = clean_gov.replace(key, val)
            clean_gov = clean_gov.strip(' ,').replace(' ', '_')
            if '_' not in clean_gov:
                clean_gov = f'NO_{clean_gov}'
            elif clean_gov.split('_')[0] in {'что', 'если', 'как', 'чтобы', 'или'}:
                filtered_out.append(clean_gov)
                continue
        elif 'из А3' in clean_gov:
            clean_gov = 'из_Gen'
        else:
            filtered_out.append(clean_gov)
            continue
        if clean_gov == 'Nom,_Acc':
            clean_gov_list.append('NO_Nom')
            clean_gov_list.append('NO_Acc')
        elif clean_gov == 'Nom_и_Nom':
            clean_gov_list.append('NO_Nom')
        elif clean_gov == 'с_Gen_до_Gen':
            clean_gov_list.append('с_Gen')
            clean_gov_list.append('до_Gen')
        elif clean_gov == 'на_Acc_и_Acc':
            clean_gov_list.append('на_Acc')
        elif clean_gov == 'или_на_Loc':
            clean_gov_list.append('на_Loc')
        elif clean_gov == 'ц_Gen':
            clean_gov_list.append('с_Gen')
        else:
            clean_gov_list.append(clean_gov)
    act_gov_models[verb] = sorted(list(set(clean_gov_list)))

In [89]:
act_gov_models['агитировать']

['NO_Acc', 'NO_Nom', 'за_Acc']

In [90]:
len(filtered_out)

858

In [91]:
filt_act_models = {}
for verb in act_gov_models:
    filt_act_models[verb] = set(act_gov_models[verb]).difference(other_case_set)

In [92]:
act_models_sum = sum([len(filt_act_models[verb]) for verb in filt_act_models])
print(f'Моделей управления в Активном словаре: {act_models_sum}')

Моделей управления в Активном словаре: 2376


In [93]:
# среднее число моделей на глагол в Активном словаре
round(act_models_sum / len(filt_act_models), 2)

2.28

In [94]:
joined_act_models = set()
for verb in filt_act_models:
    models_with_verbs = [verb + '_' + model for model in filt_act_models[verb]]
    joined_act_models.update(models_with_verbs)

### Словарь глагольного управления

In [99]:
parts = {'verb_government/' + file: round(os.stat('verb_government/' + file).st_size / 1024**2, 2)
         for file in os.listdir('verb_government')
         if file.endswith('.pickle')}
parts = dict(sorted(parts.items(), key=lambda item: item[1], reverse=True))

In [100]:
num_combos = 0
sgo_models_dict = {}
# итерируемся по файлам с частями словаря
for part in tqdm(list(parts.keys())):
    print(f'Loading {part}...')
    with open(part, 'rb') as inputfile:
        part_dict = pickle.load(inputfile)
        print(f'Successfully loaded {part}!')
        # итерируемся по глаголам в файле
        for verb in part_dict:
            v_models_set = set()
            # итерируемся по группам, с которыми сочетается глагол
            for mod_tuple in part_dict[verb]:
                v_models_set.update(mod_tuple)
            # записываем информацию в общий словарь
            if verb not in sgo_models_dict:
                sgo_models_dict[verb] = v_models_set
            else:
                sgo_models_dict[verb].update(v_models_set)

  0%|          | 0/1 [00:00<?, ?it/s]

Loading verb_government/N1_government.pickle...
{'вызывать': {('NO_Nom', 'NO_Acc', 'у_Gen'): {'рост,опасение,руководитель': [1, '0-0']}, ('NO_Acc',): {'бригада': [1, '0-4831'], 'озабоченность': [1, '0-25306'], 'выделение': [1, '0-83649'], 'количество': [1, '0-150658'], 'служба': [1, '0-153823'], 'помощь': [2, '0-153886'], 'заболевание': [2, '0-255788'], 'вопрос': [1, '1-137603'], 'уважение': [1, '1-183742'], 'последствие': [1, '1-196767']}, ('NO_Nom', 'NO_Acc'): {'звук,мурашка': [1, '0-11788'], 'поход,опасение': [1, '0-104579'], 'велосипедист,тревога': [1, '0-128672'], 'композиция,сочувствие': [2, '0-253257'], 'человек,уважение': [1, '0-269891'], 'подписание,вопрос': [1, '1-61121'], 'старт,интерес': [1, '1-98709']}, ('в_Loc',): {'случай': [1, '0-88263']}, ('NO_Acc', 'на_Acc'): {'врач,дом': [2, '0-108307'], 'педиатр,дом': [1, '0-187724']}, ('NO_Acc', 'на_Acc', 'по_Dat'): {'призывник,комиссия,график': [1, '0-128167']}, ('NO_Acc', 'на_Acc', 'через_Acc'): {'врач,дом,регистратура': [1, '0-1

In [ ]:
sgo_models_dict = dict(sorted(sgo_models_dict.items()))

In [ ]:
# with open('full_models_v4.pickle', 'wb') as outputfile:
#     pickle.dump(sgo_models_dict, outputfile)

In [40]:
with open('full_models_v4.pickle', 'rb') as inputfile:
    sgo_models_dict = pickle.load(inputfile)

In [41]:
len(sgo_models_dict)

33179

In [42]:
filtered_sgo_models = {}
for verb in sgo_models_dict:
    clean_verb = translit(verb.replace('ё', 'е'), 'ru')
    if clean_verb[0] in {'а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з'}:
        filtered_sgo_models[clean_verb] = sgo_models_dict[verb]

In [43]:
print(f'Глаголов на А-З в Словаре глагольного управления: {len(filtered_sgo_models)}')

Глаголов на А-З в Словаре глагольного управления: 9045


In [45]:
sgo_models_sum = sum([len(filtered_sgo_models[verb]) for verb in filtered_sgo_models])
print(f'Моделей управления в Словаре глагольного управления: {sgo_models_sum}')

Моделей управления в Словаре глагольного управления: 174705


In [46]:
# среднее число моделей на глагол в Словаре глагольного управления
round(sgo_models_sum / len(filtered_sgo_models), 2)

19.32

In [47]:
joined_sgo_models = set()
for verb in filtered_sgo_models:
    models_with_verbs = [verb + '_' + model for model in filtered_sgo_models[verb]]
    joined_sgo_models.update(models_with_verbs)

### Лексемы в СГО и Активном словаре

In [48]:
len(set(filt_act_models.keys()) - set(filtered_sgo_models.keys()))

1

In [49]:
set(filt_act_models.keys()) - set(filtered_sgo_models.keys())

{'высмаркиваться'}

### Модели управления в СГО и Активном словаре

In [95]:
len(joined_act_models - joined_sgo_models)

31

In [96]:
sorted(joined_act_models - joined_sgo_models)

['анализировать_с_точки_зрения_Gen',
 'браковать_в_качестве_Gen',
 'брюзжать_по_поводу_Gen',
 'верить_в_Gen',
 'взыскать_в_пользу_Gen',
 'возвысить_в_глазах_Gen',
 'войти_в_Gen',
 'волочь_по_Loc',
 'восстановить_против_Acc',
 'встать_на_Gen',
 'въехать_для_Loc',
 'выбежать_навстречу_Dat',
 'выиграть_в_мнении_Gen',
 'выиграть_по_сравнению_с_Ins',
 'выйти_за_Gen',
 'выйти_на_Gen',
 'вымогать_под_угрозой_Gen',
 'выписать_на_адрес_Gen',
 'гореть_от_Ins',
 'гулять_по_случаю_Gen',
 'дискредитировать_в_глазах_Gen',
 'договориться_о_Ins',
 'договориться_по_вопросу_Gen',
 'допустить_по_результатам_Gen',
 'достичь_через_Acc',
 'драть_об_Acc',
 'дуться_из-за_Acc',
 'запастись_на_случай_Gen',
 'затрудняться_с_Loc',
 'заявлять_по_поводу_Gen',
 'злиться_на_Gen']

In [97]:
print(', '.join(sorted(joined_act_models - joined_sgo_models)))

анализировать_с_точки_зрения_Gen, браковать_в_качестве_Gen, брюзжать_по_поводу_Gen, верить_в_Gen, взыскать_в_пользу_Gen, возвысить_в_глазах_Gen, войти_в_Gen, волочь_по_Loc, восстановить_против_Acc, встать_на_Gen, въехать_для_Loc, выбежать_навстречу_Dat, выиграть_в_мнении_Gen, выиграть_по_сравнению_с_Ins, выйти_за_Gen, выйти_на_Gen, вымогать_под_угрозой_Gen, выписать_на_адрес_Gen, гореть_от_Ins, гулять_по_случаю_Gen, дискредитировать_в_глазах_Gen, договориться_о_Ins, договориться_по_вопросу_Gen, допустить_по_результатам_Gen, достичь_через_Acc, драть_об_Acc, дуться_из-за_Acc, запастись_на_случай_Gen, затрудняться_с_Loc, заявлять_по_поводу_Gen, злиться_на_Gen
